In [2]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications.vgg16 import VGG16
from PIL import Image
from tensorflow.keras import Model
from tensorflow.python.keras.engine import data_adapter
from tensorflow.python.eager import backprop
from tensorflow.python.util import compat
from tensorflow.keras.models import load_model
from tensorflow.keras import datasets, layers, models

import tensair_aux as tensair

## Define TensorFlow Model

One may create a TensorFlow model as usual but extending TensAIR instead of Model.

In [4]:
class VGG(tensair.TensAIR):
    def __init__(self):
        super(VGG, self).__init__()
        self.l1 = VGG16(weights=None, classes=200, include_top=False, input_shape=(64,64,3))
        self.l2 = Flatten()
        self.l3 = Dense(4096, activation='relu')
        self.l4 = Dropout(rate=0.5)
        self.l5 = Dense(4096, activation='relu')
        self.l6 = Dropout(rate=0.5)
        self.l7 = Dense(200, activation='softmax')

    @tf.function
    def call(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        x = self.l5(x)
        x = self.l6(x)
        x = self.l7(x)
        return x

In [8]:
#compile TensorFlow model as usual
loss = 'sparse_categorical_crossentropy'
optimizer = 'sgd'
metrics = ['accuracy']

model = VGG()
model.compile(
    loss=loss, 
    optimizer=optimizer,   
    metrics=metrics   
)

# Prepare model for deployment

To prepare the model for deployment, we initialize the TF graph making a test prediction.

At last, we initialize the gradients that will be broadcasted via TensAIR (mandatory)

In [9]:
x = tf.zeros((1, 64, 64, 3), dtype=float)

model.predict(x)

2024-03-11 14:39:52.722033: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 262ms/step


array([[0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005,
        0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005,
        0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005,
        0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005,
        0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005,
        0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005,
        0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005,
        0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005,
        0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005,
        0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005,
        0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005,
        0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005,
        0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005,
        0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005,
      

In [10]:
model.init_delta(model.trainable_weights)

# Save Model

To save the model, we simply define the dimentions and types of the input tensors used during training, a pass those (along with the model) to the tensair.define_signatures function) to obtain the signature of the functions

At last, we save the model on the desired location uwing the signatures previously obtained

In [11]:
input_tensors_dims = [[None,64,64,3],[None]]
input_tensors_types = [tf.float32,tf.int32]
input_tensors_structure = (1,1)

In [12]:
signatures = tensair.define_signatures(model, input_tensors_dims,input_tensors_types,input_tensors_structure)

In [13]:
tensair_path = os.environ.get("TENSAIR_PATH")

In [14]:
model.save(tensair_path+"/data/vgg16/vgg16_model.tf", save_format="tf", signatures=signatures)

INFO:tensorflow:Assets written to: /Users/mauro.dalleluccatosi/Documents/GitHub-personal/TensAIR/data/vgg16/vgg16_model.tf/assets


INFO:tensorflow:Assets written to: /Users/mauro.dalleluccatosi/Documents/GitHub-personal/TensAIR/data/vgg16/vgg16_model.tf/assets
